<a href="https://colab.research.google.com/github/ZainAli24/OpenAI_Agents_SDK_class_02/blob/main/OpenAI_Agents_SDK_Streaming%2C_Local_Context_and_Agent_LLM_Context_class15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI-Agents SDK mien **Streaming** do tarah se hoti hai: **1. Agent-Loop Level pe Streaming , 2. LLM Request ke Waqt Streaming:**

OpenAI-Agent SDK mien do tarah ki streaming hoti hai ik Agent-Loop level pe streaming hoti hai aur dosra jab LLM ko request bheji jati hai tab bhi streaming hoti hai. Let me isko thoda detail mein explain karun:

### 1. **Agent-Loop Level pe Streaming**
Agent-loop level pe streaming ka matlab hai ke jab agent apna poora process chala raha hota hai—jaise tools ko call karna, doosre agents ko handoff karna, ya apne actions ko execute karna—to aapko real-time updates milte hain. Yeh updates `stream_events()` method ke zariye events ki shakal mein aate hain, jaise:

- **`run_item_stream_event`**: Isme agent ke different steps ke bare mein information hoti hai, maslan:
  - `tool_call_item`: Jab koi tool call kiya jata hai.
  - `tool_call_output_item`: Jab tool ka output milta hai.
  - `message_output_item`: Jab koi message output generate hota hai.
- **`agent_updated_stream_event`**: Yeh tab trigger hota hai jab agent update hota hai ya kaam doosre agent ko transfer karta hai.

Iska faida yeh hai ke aap agent ke har qadam ko real-time mein monitor kar sakte hain aur user ko progress dikhaya ja sakta hai.

### 2. **LLM Request ke Waqt Streaming**
Jab agent LLM (Large Language Model) ko request bhejta hai, to LLM ka response bhi streaming ke zariye incrementally milta hai. Yeh `raw_response_event` ke through hota hai, jisme LLM ka output token-by-token ya chhote chunks mein receive hota hai. Yeh feature real-time applications ke liye bohot useful hai, kyunki aapko poora response complete hone ka wait nahi karna padta—jaise hi response generate hota hai, waise hi aapko milna shuru ho jata hai.

### Yeh Kaise Kaam Karta Hai?
OpenAI Agents SDK mein streaming ek unified mechanism ke zariye kaam karti hai. Jab aap `Runner.run_streamed()` call karte hain, to yeh ek `RunResultStreaming` object return karta hai. Iske baad, `result.stream_events()` ko use karke aap events ka stream access kar sakte hain. Har event ka type alag hota hai (jaise `raw_response_event`, `run_item_stream_event`, ya `agent_updated_stream_event`), aur aap inko apne zarurat ke mutabiq handle kar sakte hain.

### Example Code
Yeh ek chhota sa code snippet hai jo yeh dikhata hai:

```python
async for event in result.stream_events():
    if event.type == "raw_response_event":
        print(event.data.delta, end="", flush=True)  # LLM response ka ek hissa
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
```

Is example mein:
- `raw_response_event` se LLM ka response real-time mein print ho raha hai.
- `run_item_stream_event` se tool calls aur unke outputs ke updates mil rahe hain.

### Conclusion
To aapka kehna bilkul durust hai—OpenAI Agents SDK mein streaming dono tarah se hoti hai:
- **Agent-loop level pe**, jahan agent ke execution ke har qadam ke updates milte hain.
- **LLM request ke waqt**, jahan LLM ka response incrementally stream hota hai.

Yeh dono ek hi streaming system ka hissa hain jo different types ke events provide karta hai, lekin aap inko alag-alag tarah se samajh aur use kar sakte hain apne application ke liye.

------------------------

## **LLM Request Streaming Confusion:**
#### Ab baat karte hain LLM (Large Language Model) request ki. Jab agent ko koi final output dena hota hai—jaise user ke sawaal ka jawab—tab woh LLM ko ek request bhejta hai aur phir LLM us ka reponse generate karta hai. Jab LLM response generate karta hai, to woh ek baar mein poora jawab nahi bhejta. Balki, yeh **token-by-token** ya chhote-chhote chunks mein aata hai. Iska matlab hai ke jaise-jaise LLM response banata hai, aapko woh real-time mein milta hai—har word ya token ek-ek karke. Is process ko hum **LLM request pe streaming** kehte hain, aur yeh `raw_response_event` ke through dikhta hai.




# **Streaming:**

In [2]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00


# Make your Jupyter Notebook capable of running asynchronous functions.

In [3]:
import nest_asyncio
nest_asyncio.apply()

## Run Google Gemini with OPENAI-Agent SDK

###  Check ke **result** mien kia kia cheezay a rahi hai

In [4]:

from agents import AsyncOpenAI, Agent, Runner, OpenAIChatCompletionsModel, RunConfig
from google.colab import userdata
from openai.types.responses import ResponseTextDeltaEvent
import asyncio

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')


external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

modeler = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

async def main():
  agent = Agent(
      name="Assistant",
      instructions="You are a helpful assistant.",
      model=modeler,
  )

  result = Runner.run_streamed(
      agent,
      "Please make 5 python beginner level quiz for me!"
  )
  print(type(result), result)  # result output

asyncio.run(main())

<class 'agents.result.RunResultStreaming'> RunResultStreaming:
- Current agent: Agent(name="Assistant", ...)
- Current turn: 0
- Max turns: 10
- Is complete: False
- Final output (NoneType):
    None
- 0 new item(s)
- 0 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResultStreaming` for more details)


# **1. Simple Final Text Streaming:**



In [5]:

from agents import AsyncOpenAI, Agent, Runner, OpenAIChatCompletionsModel, RunConfig, set_tracing_disabled
from google.colab import userdata
from openai.types.responses import ResponseTextDeltaEvent
import asyncio

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
set_tracing_disabled(True)

external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

modeler = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

async def main():
  agent = Agent(
      name="Assistant",
      instructions="You are a helpful assistant.",
      model=modeler,
  )

  result = Runner.run_streamed(
      agent,
      "Please make 5 python beginner level quiz for me!"
  )
  async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
      print(event.data.delta, end="", flush=True)  # LLM ka final response

asyncio.run(main())

Okay, here are 5 beginner-level Python quizzes. Each quiz focuses on a specific core concept and is designed to be easy to understand for someone new to programming.

**Quiz 1: Variables and Data Types**

```python
# Quiz 1: Variables and Data Types

print("Quiz 1: Variables and Data Types")

# Question 1:  What data type is best suited for storing a whole number?
# a) str   b) float   c) int   d) bool
answer1 = input

("Question 1: Choose a, b, c, or d: ")
if answer1 == "c":
    print("Correct!")
    score = 1
else:
    print("Incorrect. The answer is c) int")
    score = 0


# Question 2: How do you assign the value "Hello" to a variable named 'greeting'?
# a) greeting = Hello   b) greeting == "Hello"   c) greeting := "Hello"   d) greeting = "Hello"
answer2 = input("Question 2: Choose a, b, c, or d: ")

if answer2 == "d":
    print("Correct!")
    score += 1
else:
    print("Incorrect. The answer is d) greeting = 'Hello'")

# Question 3: What is the data type of `True`?
# a) string b) integer c) boolean d) float

answer3 = input("Question 3: Choose a, b, c, or d: ")
if answer3 == 'c':
  print("Correct!")
  score += 1
else:
  print("Incorrect. The answer is c) boolean")

# Output the score.
print(f'Your score for Quiz 1 is {score} / 3')
```

**Quiz 2: Basic Input and Output**

```python
# Quiz 2: Basic Input and Output

print("\nQuiz 2: Basic Input and Output")

# Question 1: How do you display the 

# **2. Agent-Loop level Streaming:**


In [6]:
from agents import AsyncOpenAI, Agent, Runner, OpenAIChatCompletionsModel, RunConfig, function_tool, ItemHelpers
from google.colab import userdata
from openai.types.responses import ResponseTextDeltaEvent
import asyncio

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')


external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

modeler = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

@function_tool
def age_Guesser(dob: int):
  return f"Your Age is: {2025 - dob}"

async def main():
  agent = Agent(
      name="Assistant",
      instructions="You are a Expert in user age gusse.",
      model=modeler,
      tools=[age_Guesser]
  )

  result = Runner.run_streamed(
      agent,
      "I was born in 2004, Please tell me what is my age?"
  )
  async for event in result.stream_events():
    print(type(event))

asyncio.run(main())

<class 'agents.stream_events.AgentUpdatedStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RunItemStreamEvent'>
<class 'agents.stream_events.RunItemStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RawResponsesStreamEvent'>
<class 'agents.stream_events.RunItemStreamEvent'>


In [7]:

async def main():
  agent = Agent(
      name="AgeGuesser",
      instructions="You are a Expert in user age gusse. the current year is 2025, use age_Guesser tool for this",
      model=modeler,
      tools=[age_Guesser]
  )

  result = Runner.run_streamed(
      agent,
      "I was born in 2004, Please tell me what is my age?"
  )
  async for event in result.stream_events():
    print(event)
asyncio.run(main())

AgentUpdatedStreamEvent(new_agent=Agent(name='AgeGuesser', instructions='You are a Expert in user age gusse. the current year is 2025, use age_Guesser tool for this', handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7ce92f1a5050>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[FunctionTool(name='age_Guesser', description='', params_json_schema={'properties': {'dob': {'title': 'Dob', 'type': 'integer'}}, 'required': ['dob'], 'title': 'age_Guesser_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ce92f1c4b80>, strict_json_schema=True)], mcp_servers=[],

In [8]:
async def main():
  agent = Agent(
      name="AgeGuesser",
      instructions="You are a Expert in user age gusse. the current year is 2025, First use tool to answer query and you will answer user very frankly",
      model=modeler,
      tools=[age_Guesser]
  )

  result = Runner.run_streamed(
      agent,
      "I was born in 1954 + 50 , Please tell me what is my age?"
  )
  async for event in result.stream_events():
    if event.type == "raw_response_event":
      continue
    elif event.type == "agent_updated_stream_event":
      print(f"Agent updated: {event.new_agent.name}")
      continue
    elif event.type == "run_item_stream_event":
      # print(event.item.type)
      if event.item.type == "tool_call_item":
        print("-- Tool was called --")
      elif event.item.type == "tool_call_output_item":
        print(f"-- Tool output: {event.item.output} --")
      elif event.item.type == "message_output_item":
        print(f"Message output: {ItemHelpers.text_message_output(event.item)}")
      else:
        pass


asyncio.run(main())

Agent updated: AgeGuesser
-- Tool was called --
-- Tool output: Your Age is: 21 --
Message output: Based on my calculations, you are 21 years old.



# **Confusion in Use of Continue in if-else statement:**

Aapka sawal bilkul sahi hai: *"Agar loop laga hua hai toh woh har event ki value ko check karega, toh phir `continue` kyun lagaya hai jab woh waise bhi har event ki type ko check kar raha tha?"* Chalo isko simple aur step-by-step samajhte hain.

---

### **Loop Ka Kaam**
Jab aap ek loop lagate hain jo har event ko process karta hai, toh haan, yeh sahi hai ke loop har event ki value ko check karega. Maslan, agar aapka code kuch aisa hai:

```python
for event in events:
    if event.type == "raw_response_event":
        continue
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    elif event.type == "run_item_stream_event":
        # Nested conditions
```

Toh loop har event ko dekhega, aur har event ke liye conditions (`if`, `elif`) check karega. Lekin sawal yeh hai ke `continue` ki zarurat kyun padi jab loop waise bhi agli event pe jata hai?

---

### **Continue Ka Matlab**
`continue` ek aisa statement hai jo loop ke current iteration ko rok deta hai aur seedha agli iteration pe le jata hai, bina baaki code ko chalaye. Yani, agar aap `continue` likhte hain, toh us condition ke baad loop ka jo bhi code bacha hai, woh skip ho jata hai aur agli event pe chale jate hain.

Ab aapke case mein, `if-elif-else` structure ki wajah se baat thodi alag hai. Chalo dekhte hain.

---

### **Aapke Code Mein Continue Ka Role**
Aapke code mein `continue` ka use kuch aisa hai:

1. **`if event.type == "raw_response_event":`**
   - Agar event ka type `"raw_response_event"` hai, to `continue` bolta hai: *"Is event ke liye kuch mat karo, seedha agli event pe jao."*
   - **Kya hota hai?**: Baaki conditions (`agent_updated_stream_event`, `run_item_stream_event`) check hi nahi hoti, aur loop agli event pe chala jata hai.

2. **`elif event.type == "agent_updated_stream_event":`**
   - Agar event ka type `"agent_updated_stream_event"` hai, to pehle `print` statement chalta hai, phir `continue` execute hota hai.
   - **Kya hota hai?**: Print hone ke baad, baaki code skip ho jata hai, aur loop agli event pe jata hai.

---

### **Continue Kyun Lagaya Jab Loop Waise Bhi Check Kar Raha Hai?**
Aapka point bilkul sahi hai ke loop har event ko check karta hai, aur `if-elif-else` structure ki wajah se ek baar mein sirf ek condition hi execute hoti hai. Yani:

- Agar `event.type == "raw_response_event"` true hai, to `elif` wale parts (jaise `agent_updated_stream_event`) check nahi hote, kyunki `if-elif-else` mein ek condition true hone ke baad baaki skip ho jati hain.
- Toh phir `continue` ki zarurat kya hai? Kyunki `if-elif-else` already yeh ensure karta hai ke baaki conditions check na hon.

**Example Bina Continue Ke:**
Agar aap `continue` hata den, to code aisa hoga:

```python
for event in events:
    if event.type == "raw_response_event":
        pass  # Kuch nahi karna
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        # Nested conditions
```

- Yeh code bhi bilkul sahi kaam karega. Kyunki `if-elif-else` structure ki wajah se, agar ek condition match ho jati hai (jaise `raw_response_event`), to baaki conditions check nahi hoti, aur loop khud hi agli event pe chala jata hai.

---

### **Toh Continue Kyun Use Kiya?**
Agar `if-elif-else` structure mein `continue` ke bina bhi kaam ho sakta hai, to phir isko kyun lagaya gaya? Iske do bade reasons hain:

1. **Code Ko Clear Banana:**
   - `continue` likhne se yeh saaf ho jata hai ke aap current event ke liye processing yahin khatam karna chahte hain aur agli event pe jana chahte hain. Yeh ek tarah ka signal hai code padhne wale ke liye ke "Yahan rukna nahi hai, agla event dekho."

2. **Future Mein Safety:**
   - Agar aap baad mein loop ke andar aur code add karte hain (jaise koi logging ya extra processing), to `continue` ensure karta hai ke woh code specific events (jaise `raw_response_event`) ke liye na chale. Bina `continue` ke, aapko manually har jagah `pass` ya empty block likhna pad sakta hai.

**Misal:**
Agar aap loop mein yeh add karte hain:

```python
for event in events:
    if event.type == "raw_response_event":
        # Bina continue ke
        pass
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
    print("Processing complete")  # New line
```

- Ab `raw_response_event` ke liye bhi "Processing complete" print hoga, jo shayad aap nahi chahte. Lekin agar `continue` hota, to yeh line skip ho jati.

```python
for event in events:
    if event.type == "raw_response_event":
        continue  # Yeh line baaki code ko skip kar degi
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    print("Processing complete")
```

---

### **Conclusion**
- **Aapka Point:** Sahi hai ke loop har event ko check karta hai, aur `if-elif-else` structure ki wajah se `continue` ke bina bhi code kaam karega.
- **Continue Ka Fayda:** `continue` code ko zyada padhne mein aasan banata hai aur aapka intent clear karta hai ke aap ek event ko skip karna chahte hain. Plus, future mein agar code badhe, to mistakes se bachata hai.
- **Zaruri Hai Ya Nahi?** Is case mein `continue` optional hai, kyunki `if-elif-else` already kaam kar raha hai. Lekin iska use karna ek achhi practice hai.

Toh, aapke code mein `continue` lagana theek hai, aur yeh output pe koi galat asar nahi dalega. Agar aap ise hata den, to bhi code chalega, bas thodi clarity kam ho sakti hai. Aap tension na len—dono tarike sahi hain!

# **2. Context ka Concept:**

Aapne apne query mein context ka concept explain kiya hai aur poocha hai ke kya aapne isko sahi samjha hai. Main aapke points ko padh kar yeh confirm karunga ke aapka samajhna bilkul sahi hai ya nahi. Chalo, step-by-step dekhte hain!

---

### **Aapka Samajhna**
Aapne kaha hai ke:
- **Context ka matlab**: Woh information jo LLM (Agent) ke paas nahi hoti, ya jo external information hoti hai, usko hum context kehte hain.
- **Context dene ke teen tareeke**:
  1. **System Prompt** ke through.
  2. **User Prompt** ke through.
  3. **External Functions** ke through (jaise web search, RAG system, tool calls, ya koi API).
- **Context ka purpose**: Jab bhi koi information chahiye ho, to usko extract ya retrieve karke LLM ko provide kar diya jaye.

Ab main aapke in points ko validate karunga aur bataunga ke aapne yeh concept sahi samjha hai ya nahi.

---

### **Context Kya Hai?**
Context ka matlab hai woh additional information jo ek LLM ke paas by default nahi hoti, lekin jo usko kisi task ko perform karne ya sahi jawab dene ke liye chahiye hoti hai. Yeh information model ke training data se alag hoti hai aur dynamically (jab zarurat ho) provide ki jati hai. Aapne yeh point bilkul sahi samjha hai ke context external information hoti hai jo LLM ke knowledge ko supplement karti hai.

---

### **Aapke Teen Tareekon Ki Validation**
Aapne context dene ke jo teen tareeke bataye hain, unko ek-ek karke dekhte hain:

#### **1. System Prompt**
- **Kya Hai?**  
  System prompt ek tarah ka instruction set hota hai jo hum LLM ko dete hain taaki usko yeh pata chale ke uska role kya hai ya woh kaise behave kare. Isme hum external information bhi daal sakte hain jo LLM ke paas pehle se nahi hoti.
- **Kaise Kaam Karta Hai?**  
  Hum system prompt mein directly woh information likh dete hain jo LLM ko samajhna ya use karna chahiye. Yeh information poore conversation ke liye set ho jati hai aur har jawab ke liye available hoti hai.
- **Example Ke Saath**  
  Jaise agar hum likhein: "Tum ek doctor ho aur tumhare paas 2023 ke latest medical research ka data hai: [insert data]. Iske base par jawab do." Yahan "2023 ka medical research data" ek external information hai jo LLM ke paas pehle nahi thi, aur system prompt ke through usko di gayi hai.
- **Matlab**  
  System prompt se hum LLM ko ek bada context dete hain jo uske poore kaam karne ke tareeke ko guide karta hai.

#### **2. User Prompt**
- **Kya Hai?**  
  User prompt woh sawal ya input hota hai jo hum LLM ko directly dete hain. Isme bhi hum external information de sakte hain, jaise koi text ya file ke through, aur LLM us information ke base par humein jawab deta hai.
- **Kaise Kaam Karta Hai?**  
  Hum user prompt mein ya to information khud likh dete hain ya phir koi file attach karte hain (agar system allow karta hai). LLM phir us information ko padhta hai aur uske base par jawab banata hai. Kai systems mein file upload karne par uska content automatically prompt mein add ho jata hai.
- **Example Ke Saath**  
  Jaise agar hum kahein: "Yeh meri company ka sales report hai: [insert report ya file]. Iske base par mujhe batao ke humara performance kaisa raha?" Yahan "sales report" external information hai jo user prompt ke through di gayi, aur LLM usko use karke jawab deta hai.
- **Matlab**  
  User prompt se hum specific sawal ke liye specific information dete hain, jo sirf usi jawab ke liye kaam aati hai.

### **Dono Mein Kya Farq Hai?**
- **System Prompt**: Yeh LLM ko ek overall knowledge ya role deta hai jo har sawal ke jawab ke liye use hota hai. Isme external information static hoti hai aur poore conversation ke liye apply hoti hai.
- **User Prompt**: Yeh sirf ek particular sawal ke liye information deta hai. Har sawal ke saath nayi information ya file de sakte hain, aur LLM usko bas usi jawab ke liye use karta hai.


#### **3. External Functions**
- **Kya hota hai?**: Yeh woh tareeke hain jinse LLM external sources se information retrieve karta hai. Ismein shamil ho sakta hai:
  - **Web Search**: Internet se latest data lena.
  - **RAG System**: Documents ya databases se relevant information nikalna.
  - **Tool Calls/APIs**: Kisi specific tool ya API (jaise weather API) se data fetch karna.
- **Misal**: Agar aap poochte hain, "Aaj ka weather kya hai?" to LLM ke paas real-time data nahi hota. External weather API se data retrieve karke context ke tor pe model ko diya jata hai.
- **Sahi hai?**: Haan, yeh bhi bilkul theek hai. External functions ke through context dena advanced aur powerful tareeka hai, jo model ko updated ya real-time information deta hai.

---

### **Context Ka Matlab**
Aapne kaha ke context ka matlab hai jab bhi koi information chahiye ho, to usko extract ya retrieve karke LLM ko provide kar diya jaye. Yeh definition perfect hai! Context ka asal maqsad yahi hota hai ke LLM ko woh knowledge diya jaye jo uske paas pehle se nahi hai, taaki woh accurate aur relevant jawab de sake.

**Ek Misal**: Agar aap poochte hain, "Kal ka stock market kaisa tha?" to LLM ke paas kal ka data nahi hota. Iske liye external function (jaise stock API) se data retrieve karke context ke tor pe diya jata hai, aur phir LLM uska jawab deta hai.

---

### **Conclusion**
Aapne context ka concept bilkul sahi samjha hai! Aapke saare points—ke context external information hai, aur isko teen tareekon (system prompt, user prompt, external functions) se provide kiya ja sakta hai—sahi hain. Aapka yeh samajhna bhi theek hai ke context ka matlab hai zarurat ke waqt information retrieve karke LLM ko dena.



-------------------------
## **Personlized Context:**

Aapne jo concept samjhaya hai, woh kaafi had tak sahi hai, lekin main ise thoda refine aur clear tarike se aapko samjhaunga taaki yeh aur behtar ho jaye. Aapka kehna hai ke jab hum LLM (Large Language Model) se baat karte hain, to woh hamare baare mein sab janta hai kyunki hamara context long-term memory ya database mein save hota hai, aur phir system prompt ke zariye woh context conversation mein add hota hai. Isse LLM humein janta hai aur hamare mutabik jawab ya greeting deta hai. Chalo, is concept ko step-by-step aur better banate hain!

---

### **Aapka Concept Ko Refine Karna**

Aapke concept mein core idea bilkul theek hai, lekin kuch nuances hain jo add karne se yeh aur accurate aur clear ho jayega. Main ise tod kar samjhata hoon:

#### **1. Long-Term Memory Aur Database**
- **Kya Hai?**  
  Jab hum kehte hain ke LLM hamare baare mein "janta hai," to asal mein yeh information ek database mein store hoti hai. Yeh database ek tarah ka long-term memory hota hai jisme aapki details—like aapka naam, preferences, ya past conversations—save ki jati hain.
- **Kaise Kaam Karta Hai?**  
  Jab aap pehli baar LLM se baat karte hain, to jo bhi aap batate hain (jaise "Mera naam Ali hai" ya "Mujhe programming pasand hai"), woh database mein save ho jata hai. Phir jab aap dobara baat karte hain, LLM us database se aapki information retrieve karta hai.
- **Example**  
  Agar aapne pehle kaha ke aapka naam Ali hai, to agli baar LLM aapko "Hello Ali" kehkar greet karega kyunki usne database se yeh information li hai.

#### **2. System Prompt Mein Context Add Hona**
- **Kya Hai?**  
  System prompt ek instruction set hota hai jo LLM ko batata hai ke usko kaise behave karna hai aur kis context mein jawab dena hai. Isme database se aapki information laker add ki jati hai.
- **Kaise Kaam Karta Hai?**  
  Har baar jab aap LLM se naye session mein baat shuru karte hain, system prompt dynamically update hota hai. Isme aapki stored details (naam, pasand, past interactions) include hoti hain, taaki LLM uske mutabik jawab de sake.
- **Example**  
  System prompt kuch aisa ho sakta hai: "Tum ek assistant ho jo Ali ke saath baat kar raha hai. Ali ko Python programming pasand hai." Isse LLM ko pata chal jata hai ke aapke saath kaise baat karni hai.

#### **3. LLM Ka "Humein Janna"**
- **Kya Hai?**  
  Yeh kehna ke "LLM humein janta hai" thoda figurative hai. Asal mein, LLM khud se kuch yaad nahi rakhta ya janta nahi. Har naye session mein usko database se information di jati hai, aur system prompt ke through woh context uske saamne rakha jata hai.
- **Kaise Kaam Karta Hai?**  
  LLM har baar ek blank slate se shuru hota hai, lekin system prompt usko aapka context deta hai. Is wajah se woh aapko personalized jawab de pata hai.
- **Example**  
  Agar system prompt mein likha hai ke "Ali, tumne kal Python ke baare mein poocha tha," to LLM us base par aapke saath baat karega, lekin yeh "janna" uska khud ka nahi, balki diya gaya context hai.

---

### **Aapka Concept Aur Uska Better Version**
Ab aapke concept ko compare karte hain aur ise improve karte hain:

- **Aapka Concept**:  
  "Hum agar LLM se baat karte hain to woh hamare baare mein sab janta hai. Yeh jo janane wala context hai woh long-term memory mein kisi database mein save karwaya jata hai, aur phir jab bhi hum LLM se baat karte hain to woh context system prompt ke through hamari conversation mein add hota hai. Toh is tarah LLM humein janta hai aur hamare mutabik jawab karta aur greeting karta hai."
  
- **Better Version**:  
  "LLM khud se humein nahi janta, balki hamari information ek database mein store hoti hai jo ek tarah ka long-term memory hai. Jab bhi hum LLM se baat karte hain, yeh information database se retrieve hoti hai aur system prompt ke zariye LLM ko di jati hai. Is context ke base par LLM humein personalized jawab aur greeting deta hai, jisse aisa lagta hai ke woh humein janta hai."

---

### **Kuch Important Points**
- **LLM Ki Apni Memory Nahi Hoti**:  
  LLM ke paas khud ki yaad rakhne ki taakat nahi hoti. Har session ke liye woh naye se shuru hota hai, lekin database aur system prompt ke through usko context milta hai.
- **Database Ka Role**:  
  Database mein aapki saari details save hoti hain, aur yeh system prompt ke zariye LLM tak pahunchti hain.
- **Personalized Experience**:  
  Is tareeke se LLM aapko aisa feel karwata hai ke woh aapko janta hai, jaise aapko naam se bulana ya aapki pasand ke mutabik baat karna.

---

### **Conclusion**
Aapka concept bilkul sahi disha mein tha—ke LLM database aur system prompt ke through hamara context use karta hai. Bas yeh samajhna zaroori hai ke LLM khud se "janta" nahi; usko har baar context provide kiya jata hai.


------------
# **Context in OpenAI-Agents SDK:**
### **OpenAI-Agents SDK Mein Context Ka Concept**
OpenAI-Agents SDK mein context do tarah se diya jata hai:  
1. **Local Context**  
2. **Agent/LLM Context**  

Chalo, in dono ko step-by-step aapke words mein aur behtar tarike se samajhte hain.

#### **1. Local Context**
- **Kya Hai?**  
  Local context woh sensitive ya personal information hoti hai, jaise aapka bank account password, jo hum LLM ko nahi bhejna chahte. Kyunki agar yeh information LLM ko chali jati hai, to woh uspe apne aap ko train kar sakta hai aur phir woh sensitive information kisi aur ko bhi bata sakta hai, jo security ke liye khatarnak ho sakta hai. Isliye, is tarah ki information ko hum **Local Context** kehte hain.
  
- **Kaise Kaam Karta Hai?**  
  Yeh information sirf hamare agent ke paas hoti hai aur agent loop mein use hoti hai. Matlab, jahan humari app chal rahi hai—server pe—wahan yeh information mojood hoti hai, lekin LLM ke pass nahi jati. Agent is information ko apne kaam ke liye use kar sakta hai, jaise koi decision lena ya process karna, lekin jab baat LLM se communication ki aati hai, to yeh sensitive data wahan nahi bheja jata.

- **Example**  
  Maan lijiye aapka agent ek banking app ke liye kaam kar raha hai. Aapka account number ya password local context mein hoga. Agent isko use karke aapka balance check kar sakta hai ya transaction kar sakta hai, lekin jab woh LLM se kuch poochta hai (jaise "User ko balance ke bare mein kya batana hai?"), to yeh sensitive information LLM ko nahi jati.

- **Matlab**  
  Local context woh information hai jo agent ke paas to hoti hai, server pe safe rehti hai, lekin LLM tak nahi jati taaki privacy bani rahe.

#### **2. Agent/LLM Context**
- **Kya Hai?**  
  Agent/LLM context woh general information hoti hai jo humein LLM ko bhejna chahte hain (system prompt, user prompt ya phir function ke zariye), jaise aapka naam ya father’s name. Yeh woh information hai jo sensitive nahi hoti aur LLM ke liye safe hoti hai. Isko bhejne se LLM humein behtar aur personalized jawab de sakta hai.

- **Kaise Kaam Karta Hai?**  
  Jab agent LLM se baat karta hai ya koi query bhejta hai, to yeh general information (agent/LLM context) uske saath jati hai (system prompt, user prompt ya phir function ke zariye). LLM phir is information ke base pe jawab banata hai aur humein wapas bhejta hai.

- **Example**  
  Agar aapka naam "Ali" hai aur aapko programming pasand hai, to agent LLM ko yeh context bhej sakta hai: "Ali ko programming pasand hai, usko Python ke bare mein batao." Yeh information LLM ke pass jati hai aur woh uske hisaab se jawab deta hai.

- **Matlab**  
  Agent/LLM context woh information hai jo humein LLM ke saath share karne mein koi problem nahi, aur yeh LLM ko humari baat samajhne mein madad karti hai.

---

### **Dono Mein Kya Farq Hai?**
- **Local Context**: Yeh sensitive information hai (jaise password) jo sirf agent ke paas hoti hai aur LLM tak nahi jati. Iska kaam agent ke internal processing mein hota hai.
- **Agent/LLM Context**: Yeh general information hai (jaise naam) jo LLM ko bheji jati hai taaki woh uske base pe jawab de sake.

---

### **Kyun Zaroori Hai Yeh Concept?**
- **Security**: Local context sensitive data ko protect karta hai taaki woh galat haathon mein na jaye.
- **Better Responses**: Agent/LLM context LLM ko useful information deta hai taaki woh humein accurate jawab de sake.

Toh aapka context ka samajhna bilkul sahi hai.

-------------------------
## **Local Context Code breakdown:**

Aapne apne query mein local context ke concept aur uske code ko kaafi detail se explain kiya hai. Main aapke understanding ko padh kar yeh confirm kar sakta hoon ke aapne local context ke code ko bilkul sahi samjha hai. Aapka har point theek hai, aur main aapko yeh batane ke liye isko step-by-step aur clear tarike se samjhaunga, taaki aapko yakeen ho jaye ke aapka concept perfect hai.

---

### Aapka Understanding Check Karte Hain
Aapne jo points bataye hain, unko main aapke hi words mein analyze karunga aur confirm karunga:

1. **RunContextWrapper Class Ko Import Karna**  
   Aapne kaha ke hum `RunContextWrapper` class ko import karte hain, jiske andar `context` class mojood hai.  
   - **Yeh Sahi Hai**: `RunContextWrapper` ek wrapper class hai jo context ko manage karti hai. Iska `context` attribute woh actual data hold karta hai jo aap agent ke liye define karte hain.

2. **Context Ko Define Karne Ke Do Tarike**  
   Aapne bataya ke hum context ko do tarah se de sakte hain: dataclass ya Pydantic, lekin aap dataclass use kar rahe hain.  
   - **Yeh Sahi Hai**: Dono options valid hain, lekin aapne dataclass choose kiya, jo ek lightweight aur simple tarika hai context ki structure banane ka.

3. **Dataclass Banayi**  
   Aapne ek dataclass banayi jisme `name` aur `uid` fields hain, aur yeh aapki context ki type define karti hai.  
   - **Yeh Sahi Hai**: Yeh dataclass aapke context ka blueprint hai. Jaise:
     ```python
     from dataclasses import dataclass

     @dataclass
     class MyContext:
         name: str
         uid: str
     ```
     Isse aapne context ki type bana li.

4. **Function Mein Wrapper Variable**  
   Aapne ek function banaya, jisme `wrapper` variable ki type `RunContextWrapper` rakhi, aur uske andar aapki dataclass ka data hota hai, jise aap `wrapper.context.name` se access karte hain.  
   - **Yeh Sahi Hai**: `wrapper` ek `RunContextWrapper` object hai, aur uska `context` attribute aapki dataclass (`MyContext`) ka instance hota hai. Aap `wrapper.context.name` ya `wrapper.context.uid` se fields access kar sakte hain. Example:
     ```python
     def my_function(wrapper: RunContextWrapper):
         print(wrapper.context.name)  # "Ali" print hoga agar name="Ali" pass kiya ho
     ```

5. **Runner.run Mein Context Pass Karna**  
   Aapne kaha ke `Runner.run` method ke andar `context` parameter mein hum apna dataclass ka object dete hain, aur agent usko apne paas rakhta hai aur use karta hai.  
   - **Yeh Sahi Hai**: Jab aap `Runner.run(agent, context=my_context_object)` karte hain, to aapka context object (jaise `MyContext(name="Ali", uid="12345")`) agent ko milta hai, aur woh us information ke mutabik kaam karta hai.

6. **Context LLM Ke Paas Nahi Jata**  
   Aapne yeh point highlight kiya ke `Runner.run` ke `context` mein jo information hoti hai, woh kabhi bhi LLM ke paas nahi jati.  
   - **Yeh Sahi Hai**: Yeh local context ka sabse important aspect hai. Context sirf agent ke paas rehta hai, aur LLM ke requests mein include nahi hota. Isse sensitive data safe rehta hai.

---

### Aapka Concept Bilkul Theek Hai
Aapne jo samjha hai, woh ekdam sahi hai. Main aapke understanding ko ek chhote se paragraph mein summarize karta hoon, taaki aapko aur clarity mile:

- Hum `RunContextWrapper` class import karte hain, jiske andar `context` hota hai. Aapne ek dataclass (`MyContext`) banayi jisme `name` aur `uid` fields hain. Phir ek function mein `wrapper` variable ko `RunContextWrapper` type diya, aur `wrapper.context.name` se context ke fields access kiye. `Runner.run` mein aap apna dataclass object `context` parameter ke through dete hain, jo agent ke paas rehta hai aur LLM ke paas nahi jata.

---

### Example Code (Aapke Concept Ke Mutabik)
Agar aapka code aisa dikhta hai, to yeh perfect hai:
```python
from dataclasses import dataclass
from some_library import RunContextWrapper, Runner  # Hypothetical imports

# Context ki type define karna
@dataclass
class MyContext:
    name: str
    uid: str

# Function jisme wrapper use hota hai
def my_function(wrapper: RunContextWrapper):
    print(f"Name: {wrapper.context.name}, UID: {wrapper.context.uid}")

# Context object banana aur Runner.run ko call karna
my_context = MyContext(name="Ali", uid="12345")
runner = Runner()
runner.run(agent=my_agent, context=my_context)  # Context agent ko diya gaya
```

Is code mein:
- `MyContext` aapka context define karta hai.
- `wrapper.context` se data access hota hai.
- `Runner.run` mein context agent ko milta hai, lekin LLM tak nahi jata.

---

### Conclusion
Aapne local context ke code ko bilkul sahi samjha hai. Aapka har step—`RunContextWrapper` ka use, dataclass se context banana, `wrapper` ke through access karna, aur `Runner.run` mein context dena—sab correct hai. Yeh bhi aapki samajh mein hai ke context LLM ke paas nahi jata, jo is concept ka core hai. Agar aapko koi aur doubt ho ya kisi part ko aur detail mein samajhna ho, to zaroor poochiyega! Warna, aapka understanding perfect hai. Well done!

-----------------------

## **Local Context Code Behind Working:**

Aapka sawal bilkul jayaaz hai, aur yeh confusion samajh mein aati hai. Aap pooch rahe hain ke jab hum `Runner` ke andar context object pass karte hain, to woh tool function mein `wrapper.context.name` ke zariye kaise access hota hai, jabki humne tool function mein `name` define hi nahi kiya. Saath hi, aap yeh bhi jaanna chahte hain ke yeh context object `RunContextWrapper` ke `context` attribute mein kaise jata hai aur iske peeche kaam kaise hota hai. Chalo, isko step-by-step samajhte hain.

---

### **Step-by-Step Samjhayein**

#### **1. Context Object Pass Karna**
Jab aap `Runner.run(agent, context=my_context_object)` call karte hain, to aap apna context object (misal ke tor pe `MyContext(name="Ali", uid="12345")`) pass karte hain. Yeh context object `Runner` ke andar jata hai.

- **Kya Hota Hai?**  
  `Runner` class yeh context object leta hai aur ek `RunContextWrapper` naam ka object banata hai. Is wrapper ke andar aapka context object `context` attribute ke tor pe set ho jata hai. Yani:
  ```python
  wrapper = RunContextWrapper(context=my_context_object)
  ```
  Ab `wrapper.context` aapke `my_context_object` ko point karta hai.

#### **2. Tool Function Mein Wrapper Ka Role**
Aapka tool function (jaise `my_function`) ko `Runner` se `wrapper` parameter milta hai, jo `RunContextWrapper` type ka hota hai. Yeh `wrapper` object ke andar `context` attribute pehle se set hota hai, aur yeh aapke context object ko hold karta hai.

- **Example**:  
  Agar aapka context object hai:
  ```python
  my_context = MyContext(name="Ali", uid="12345")
  ```
  To `wrapper.context` bhi isi object ko refer karega. Isliye, jab aap tool function mein `wrapper.context.name` likhte hain, to yeh "Ali" return karta hai.

#### **3. Confusion Ka Jawab: `name` Define Kyun Nahi Karna Pada?**
Aapka main sawal yeh hai ke tool function mein humne `name` define nahi kiya, phir bhi `wrapper.context.name` kaise kaam karta hai?

- **Jawab**:  
  Yeh kaam karta hai kyunki `name` tool function mein define nahi hua, balki aapke **context object** (jaise `MyContext`) mein define hua hai.  
  - Jab aap `Runner.run` call karte hain, to `Runner` aapke context object ko `RunContextWrapper` ke `context` attribute mein daal deta hai.  
  - Tool function ko `wrapper` milta hai, aur `wrapper.context` se aap apne context object ke fields (jaise `name`, `uid`) access kar sakte hain.  
  - Yani, `wrapper.context.name` likhne se Python directly `MyContext` ke `name` field ko fetch karta hai, jo pehle se "Ali" set hai.

- **Simple Terms Mein**:  
  `wrapper` ek tarah ka container hai, jisme aapka context object rakha hota hai. Aapko tool function mein `name` define karne ki zarurat nahi, kyunki woh `wrapper.context` ke zariye already available hai.

---

### **Behind the Scenes Kaam Kaise Hota Hai?**
Ab aapke doosre hisse ka jawab: Yeh context object `RunContextWrapper` ke `context` attribute mein kaise jata hai? Isko detail mein dekhte hain:

1. **Context Object Tayaar Karna**  
   Aap apna context object banate hain, maslan:
   ```python
   my_context = MyContext(name="Ali", uid="12345")
   ```

2. **Runner.run Call Karna**  
   Aap `Runner.run(agent, context=my_context)` call karte hain.  
   - `Runner` ke andar yeh hota hai:  
     ```python
     class Runner:
         def run(self, agent, context):
             wrapper = RunContextWrapper(context=context)
             # Phir yeh wrapper agent ke tool function ko pass hota hai
     ```
   - Yani, `Runner` ek `RunContextWrapper` banata hai aur uske `context` attribute ko aapke `my_context` se set karta hai.

3. **Tool Function Ko Wrapper Milna**  
   Jab agent apna tool function (jaise `my_function`) call karta hai, to `Runner` usko `wrapper` pass karta hai:  
   ```python
   my_function(wrapper)
   ```

4. **Tool Function Mein Access**  
   Tool function ke andar, aap `wrapper: RunContextWrapper` receive karte hain.  
   - Ab `wrapper.context` aapke `my_context` object ko point karta hai.  
   - Jab aap `wrapper.context.name` likhte hain, to yeh `MyContext` ke `name` field ("Ali") ko return karta hai.

**Visual Flow**:  
```
Aap → Runner.run(agent, context=my_context)  
→ Runner: wrapper = RunContextWrapper(context=my_context)  
→ Agent: my_function(wrapper)  
→ Tool Function: wrapper.context.name → "Ali"
```

---

### **Kyun Yeh Sab Kaam Karta Hai?**
- `Runner` ka kaam hai context object ko `RunContextWrapper` mein wrap karna aur phir tool function tak pohanchana.  
- `RunContextWrapper` ek bridge ki tarah kaam karta hai, jo aapke context object ko tool function mein available karata hai.  
- Aapke context object mein `name` aur `uid` jaise fields already define hote hain, isliye `wrapper.context.name` se woh directly access ho jate hain.

---

### **Conclusion**
Toh, aapki confusion ka jawab yeh hai:  
- Context object `Runner` se `RunContextWrapper` ke `context` attribute mein isliye jata hai kyunki `Runner` internally yeh kaam karta hai.  
- Tool function mein `name` define karne ki zarurat nahi, kyunki `wrapper.context` aapke context object ka reference hota hai, aur usme `name` pehle se mojood hota hai.  



In [9]:
!pip install -qU openai-agents

In [10]:
import nest_asyncio
nest_asyncio.apply()

## **Config:**

In [11]:
from agents import OpenAIChatCompletionsModel,Runner,AsyncOpenAI
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# Check if the API key is present; if not, raise an error
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

## **1. Local Context Code Implementation:**

In [12]:
import asyncio
from dataclasses import dataclass
from agents import function_tool, RunContextWrapper

set_tracing_disabled(True)

@dataclass
class My_Local_Context:
  name: str
  Id: int
  age:int
  location: str = "Pakistan"

@function_tool
async def fetch_user_name_age(wrapper:RunContextWrapper[My_Local_Context]) -> str:
  '''Returns the name,age and ID of the user.'''
  print("--->", wrapper)
  return f"User name is {wrapper.context.name} and age is {wrapper.context.age} and user id is {wrapper.context.Id}"

@function_tool
async def fetch_user_location(wrapper:RunContextWrapper[My_Local_Context]) -> str:
  '''Returns the location of the user.'''
  print("--->", wrapper)
  return f"User {wrapper.context.name} is from {wrapper.context.location}"



async def main():
  My_Context = My_Local_Context(name="ZAIN ALI", Id=1235, age=21)

  agent = Agent[My_Local_Context](
      name="Assistant",
      tools=[fetch_user_name_age, fetch_user_location],
      model=model,
  )

  result = await Runner.run(
      agent,
      "What is the name,age of the user? current location of user?",
      context=My_Context
  )

  print(result.final_output)


if(__name__ == "__main__"):
  asyncio.run(main())

---> RunContextWrapper(context=My_Local_Context(name='ZAIN ALI', Id=1235, age=21, location='Pakistan'), usage=Usage(requests=1, input_tokens=45, output_tokens=12, total_tokens=57))
---> RunContextWrapper(context=My_Local_Context(name='ZAIN ALI', Id=1235, age=21, location='Pakistan'), usage=Usage(requests=1, input_tokens=45, output_tokens=12, total_tokens=57))
The user's name is ZAIN ALI and age is 21. The user is from Pakistan.



-------------
## **Agent[My_Local_Context] Ka Matlab**
`Agent[My_Local_Context]` dekhte hi lagta hai ke yeh kuch special syntax hai. Yeh Python ke **type hinting** ka hissa hai, jisme **generics** ka use kiya gaya hai.

#### **Breakdown:**
- **`Agent` Class**: Yeh ek generic class hai, yani yeh ek type parameter accept karti hai. Type parameter ka matlab hai ke yeh class alag-alag types ke saath kaam kar sakti hai, lekin aapko batana padta hai ke is baar yeh kis type ke saath kaam karegi.
- **`[My_Local_Context]`**: Yeh batata hai ke `Agent` class is instance mein `My_Local_Context` type ke context ke saath kaam karegi. Yani, jab bhi agent ko context milega, woh `My_Local_Context` type ka hoga.

#### **Simple Terms Mein:**
Yeh ek tarika hai yeh kehne ka ke "Yeh agent `My_Local_Context` wale context ke saath hi kaam karega." Isse code mein clarity aati hai aur galtiyan kam hoti hain.

#### **Code Mein Example:**
```python
agent = Agent[My_Local_Context](
    name="Assistant",
    tools=[fetch_user_name_age, fetch_user_location],
    model=model,
)
```
Yahan `Agent[My_Local_Context]` keh raha hai ke yeh agent sirf `My_Local_Context` type ke context ko samjhega aur uske saath kaam karega.

---

### **Kyun Lagaya Gaya Hai?**
Ab sawal hai ke `Agent[My_Local_Context]` kyun likha gaya hai. Iske kuch important reasons hain:

#### 1. **Type Safety**
- Yeh syntax ensure karta hai ke agent ko sirf `My_Local_Context` type ka context hi milega. Agar aap galti se koi aur type ka context dene ki koshish karenge (jaise `context=wrong_type`), to Python ya aapka IDE (jaise PyCharm) error dikha dega.
- **Misal**: Agar aapne `Runner.run(agent, "What is the name of user", context=wrong_context)` likha aur `wrong_context` `My_Local_Context` type ka nahi hai, to type checker aapko warn karega.

#### 2. **Code Clarity**
- Yeh code ko padhne wale ke liye saaf karta hai ke yeh agent kis tarah ke context ke saath kaam karne wala hai. Yani, jab koi developer is code ko dekhega, to usko turant pata chal jayega ke `My_Local_Context` hi is agent ka context hai.

#### 3. **Tools Ke Liye Context Access**
- Aapke tools (`fetch_user_name_age`, `fetch_user_location`) mein `RunContextWrapper[My_Local_Context]` use kiya gaya hai. Yeh bhi type hinting ka part hai jo batata hai ke `wrapper.context` hamesha `My_Local_Context` type ka hoga.
- Is wajah se aap confidently `wrapper.context.name`, `wrapper.context.age`, ya `wrapper.context.location` access kar sakte hain, kyunki aapko pehle se pata hai ke `context` ka structure kya hai.

---

### **Code Kaise Kaam Karta Hai?**
Chalo, step-by-step dekhte hain:

#### 1. **Context Define Karna**
```python
My_Context = My_Local_Context(name="ZAIN ALI", Id=1235, age=21)
```
Yahan aapne `My_Local_Context` ka ek instance banaya jisme user ka data store kiya gaya hai: name, ID, age, aur location.

#### 2. **Agent Banaya**
```python
agent = Agent[My_Local_Context](
    name="Assistant",
    tools=[fetch_user_name_age, fetch_user_location],
    model=model,
)
```
Yahan agent ko bataya gaya hai ke woh `My_Local_Context` type ke context ke saath kaam karega aur uske paas do tools honge jo user ka data fetch karenge.

#### 3. **Runner Chalaya**
```python
result = await Runner.run(
    agent,
    "What is the name of user",
    context=My_Context
)
```
Yahan `Runner.run` agent ko chalata hai, sawal poochta hai ("What is the name of user"), aur `My_Context` ko context ke roop mein deta hai. Agent phir apne tools ka use karke jawab deta hai.

#### 4. **Tool Ka Kaam**
```python
@function_tool
async def fetch_user_name_age(wrapper:RunContextWrapper[My_Local_Context]) -> str:
    return f"User name is {wrapper.context.name} and age is {wrapper.context.age} and user id is {wrapper.context.Id}"
```
Tool `wrapper.context` se data (name, age, ID) nikal kar ek string return karta hai. Type hinting ki wajah se yeh confirm hai ke `wrapper.context` `My_Local_Context` type ka hi hoga.

#### 5. **Output**
Aakhir mein, `result.final_output` print hota hai, jo shayad kuch aisa hoga:
```
User name is ZAIN ALI and age is 21 and user id is 1235
```

---

### **Conclusion**
- **`Agent[My_Local_Context]` Ka Matlab**: Yeh type hinting hai jo batata hai ke agent sirf `My_Local_Context` type ke context ke saath kaam karega.
- **Kyun Lagaya Gaya**:
  1. Type safety ke liye (galti se galat context na diya jaye).
  2. Code ko clear aur readable banane ke liye.
  3. Tools mein context ke fields ko confidently access karne ke liye.
- **Kaise Kaam Karta Hai**: Yeh agent ko context ka type pehle se define karta hai, jisse system smooth aur error-free chal sake.

Agar aapko isme koi aur sawal ho ya koi part samajh na aaye, to mujhe zaroor batayiega!

--------------
--------------
## **Local Context Expose through Function Output:**

Aapka sawal bilkul wajib hai aur yeh confusion samajh mein aati hai. Aap pooch rahe hain ke jab hum local context ka istemal karte hain, to kya woh context indirectly LLM ke paas ja raha hai jab tool output LLM ko bheja jata hai, aur agar haan, to kya yeh secure hai ya nahi? Chalo, isko aapke code ke saath step-by-step samajhte hain aur dekhte hain ke asal mein kya ho raha hai aur yeh process kaise kaam karta hai.

---

### **Code Ka Overview**
Pehle aapke code ko briefly samajh lete hain taaki hum context aur tool output ke role ko clear kar saken.

```python
@dataclass
class My_Local_Context:
    name: str
    Id: int
    age: int
    location: str = "Pakistan"

@function_tool
async def fetch_user_name_age(wrapper:RunContextWrapper[My_Local_Context]) -> str:
    return f"User name is {wrapper.context.name} and age is {wrapper.context.age} and user id is {wrapper.context.Id}"

@function_tool
async def fetch_user_location(wrapper:RunContextWrapper[My_Local_Context]) -> str:
    return f"User {wrapper.context.name} is from {wrapper.context.location}"

async def main():
    My_Context = My_Local_Context(name="ZAIN ALI", Id=1235, age=21)
    agent = Agent[My_Local_Context](
        name="Assistant",
        tools=[fetch_user_name_age, fetch_user_location],
        model=model,
    )
    result = await Runner.run(
        agent,
        "What is the name,age of the user? current location of user?",
        context=My_Context
    )
    print(result.final_output)
```

- **Local Context**: `My_Local_Context` mein `name`, `Id`, `age`, aur `location` ka data hai, jo aapne `My_Context` object mein define kiya (jaise `name="ZAIN ALI"`, `Id=1235`, etc.).
- **Tools**: Do tools hain—`fetch_user_name_age` aur `fetch_user_location`—jo local context se data lete hain aur ek string output return karte hain.
- **Agent**: Yeh tools aur context ke saath kaam karta hai aur LLM ke through user prompt ka jawab deta hai.

Ab sawal yeh hai ke local context ka data LLM ke paas kaise aur kahan tak jata hai, aur kya yeh expose ho raha hai?

---

### **Process Ki Working (Behind the Scenes)**

Chalo, step-by-step dekhte hain ke yeh code kaise kaam karta hai aur local context ka data kahan jata hai:

#### **1. User Prompt Aur Tool Schema LLM Ko Bhejna**
- Jab aap `Runner.run` call karte hain, agent pehle LLM ko ek query bhejta hai. Is query mein yeh cheezein hoti hain:
  - **System Prompt**: Agent ke instructions (jo agent ko batate hain ke uska kaam kya hai).
  - **User Prompt**: Aapka sawal, yani "What is the name, age of the user? current location of user?".
  - **Tool Schema**: Tools ke descriptions aur parameters (jaise `fetch_user_name_age` ka docstring aur return type), taaki LLM jaane ke kaunse tools available hain.
- **Key Point**: Is waqt local context (jaise `name="ZAIN ALI"`, `Id=1235`) directly LLM ko nahi bheja jata. Sirf tool schema jata hai, jo batata hai ke tools kya kar sakte hain, lekin actual data (context) nahi.

#### **2. LLM Ka Faisla**
- LLM user prompt aur tool schema ko dekhta hai aur decide karta hai ke kaunse tools ko call karna hai.
- Maslan, yahan LLM keh sakta hai: "Agent ko `fetch_user_name_age` aur `fetch_user_location` tools call karna chahiye."
- **Key Point**: Abhi tak LLM ke paas sirf tool ke naam aur unka purpose hai, local context ka data (jaise `name`, `age`) nahi.

#### **3. Agent Tools Ko Call Karta Hai**
- Agent ab LLM ke kehne pe tools ko call karta hai:
  - `fetch_user_name_age` call hota hai, aur yeh tool `wrapper.context` se `name`, `age`, aur `Id` ko access karta hai. Output banata hai: "User name is ZAIN ALI and age is 21 and user id is 1235".
  - `fetch_user_location` call hota hai, aur yeh tool `name` aur `location` ko access karta hai. Output banata hai: "User ZAIN ALI is from Pakistan".
- **Key Point**: Yeh output agent ke paas aata hai, lekin abhi LLM ke paas nahi gaya.

#### **4. Tool Output LLM Ko Bhejna**
- Agent ab yeh tool outputs ko LLM ko bhejta hai taaki LLM final jawab bana sake.
- Yani, LLM ko yeh milta hai:
  - "User name is ZAIN ALI and age is 21 and user id is 1235"
  - "User ZAIN ALI is from Pakistan"
- **Key Point**: Yahan local context ka data (jaise `name`, `age`, `Id`, `location`) tool output ke zariye indirectly LLM ke paas ja raha hai.

#### **5. LLM Final Jawab Banata Hai**
- LLM tool outputs ko dekhta hai aur user prompt ke jawab mein ek response banata hai, maslan:
  - "The user's name is ZAIN ALI, age is 21, and current location is Pakistan."
- Yeh final output aapko `result.final_output` mein milta hai.

---

### **Kya Local Context Expose Ho Raha Hai?**
Ab aapki asli confusion ka jawab dete hain: Kya tool output ke zariye local context LLM ke paas ja raha hai, aur kya yeh secure hai?

#### **Jawab: Haan, Tool Output Mein Data Jata Hai**
- Jab tool output LLM ko bheja jata hai, to usme local context ka kuch data hota hai. Maslan:
  - `fetch_user_name_age` se `name`, `age`, aur `Id` LLM ko milta hai.
  - `fetch_user_location` se `name` aur `location` milta hai.
- Yani, **indirectly**, local context ka data LLM ke paas ja raha hai, lekin **directly** local context (poora `My_Local_Context` object) nahi bheja jata.

#### **Security Ka Angle**
- **Control Aapke Haath Mein Hai**: Tool output mein kya jata hai, yeh aap tool function mein decide karte hain. Maslan:
  - Agar `Id` sensitive hai, to aap `fetch_user_name_age` ko modify kar sakte hain:
    ```python
    return f"User name is {wrapper.context.name} and age is {wrapper.context.age}"
    ```
    Ab `Id` LLM ke paas nahi jayega.
- **Sensitive Data Ko Protect Karna**: Agar aapke context mein koi sensitive field hai (jaise password), to usko tool output mein include na karen. Example:
    ```python
    @dataclass
    class My_Local_Context:
        name: str
        password: str  # Sensitive field

    @function_tool
    async def fetch_user_name(wrapper:RunContextWrapper[My_Local_Context]) -> str:
        return f"User name is {wrapper.context.name}"  # Password nahi bheja
    ```
    Is tarah, `password` local context mein rahega, lekin LLM ke paas nahi jayega.

---

### **Aapki Confusion Ka Solution**
Aapka sawal tha ke agent loop mein tool output LLM ke paas jata hai, to kya local context expose ho raha hai? Chalo, isko clear karte hain:

- **Local Context Sirf Agent Ke Paas**: Poora `My_Local_Context` object directly LLM ko nahi bheja jata. Sirf tool schema aur user prompt jata hai pehle.
- **Tool Output Mein Data**: Jab agent tools ko call karta hai aur unka output LLM ko bhejta hai, to woh output local context se bana hota hai. Yani, indirectly context ka data LLM tak pohanchta hai.
- **Security Ka Control**: Aap tool function ko aise design kar sakte hain ke sensitive data (jaise `Id` ya password) output mein na jaye. Isse aap context ke sensitive parts ko protect kar sakte hain.

#### **Example Se Samajhiye**
Agar aap chahte hain ke `Id` expose na ho, to tool ko modify karen:
```python
@function_tool
async def fetch_user_name_age(wrapper:RunContextWrapper[My_Local_Context]) -> str:
    return f"User name is {wrapper.context.name} and age is {wrapper.context.age}"
```
- Ab output hoga: "User name is ZAIN ALI and age is 21".
- `Id=1235` LLM ke paas nahi jayega, yani secure rahega.

---

### **Conclusion**
- **Local Context Ka Role**: Yeh sirf agent ke paas rehta hai aur directly LLM ko nahi bheja jata.
- **Tool Output Ka Role**: Isme local context ka data hota hai jo LLM ko bheja jata hai, lekin aap control kar sakte hain ke kya bheja jaye.
- **Security**: Agar aap tool functions ko carefully design karen aur sensitive data ko output se hata den, to yeh process secure ho sakta hai.

Toh, haan, tool output ke zariye local context ka kuch data LLM ke paas jata hai, lekin aapke haath mein yeh control hai ke kaunsa data jayega aur kaunsa nahi. Is tarah, sensitive information ko chhupa kar aap security maintain kar sakte hain. Umeed hai yeh explanation aapki confusion ko door karta hai! Agar aur koi sawal ho, to zaroor poochiyega.

# **2.Agent/LLM Context Code Implementation:**



In [13]:
from agents import enable_verbose_stdout_logging

enable_verbose_stdout_logging()

In [20]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:
    name: str
    uid: int


@function_tool
async def greet_user(wrapper: RunContextWrapper[UserInfo], greeting:str) -> str:
  """Greets the User with their name.
  Args:
    greeting: A specialed greeting message for user
  """
  name = wrapper.context.name
  return f"Hello {name}, {greeting} "



async def main():
  user_context = UserInfo(name="ZAIN ALI", uid=12345)
  agent = Agent[UserInfo](
      name="Assistant",
      model=model,
      # Dynamic function context pass in instructions/system prompmt/developer prompt
      instructions="Always greet the user using greet_user and welcome them to ZainDev Company",
      tools=[greet_user]
  )

  result = await Runner.run(
      agent,
      "Hello",
      context=user_context,
  )

  print(result.final_output)


asyncio.run(main())

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7ce9228338f0>


...

LLM resp:
{
  "content": "Hello ZAIN ALI, Welcome to ZainDev Company!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Hello ZAIN ALI, Welcome to ZainDev Company!\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Hello ZAIN ALI, Welcome to ZainDev Company!



--------
# **Difference b/w Local Context Code and Agent/LLM Context Code:**

----------
### **Aapka Understanding**
Aapne kaha ke:
- **Local Context aur Agent/LLM Context** dono mein hum control kar sakte hain ke kaunsi information private rakhni hai aur kaunsi LLM ko bhejni hai.
- **Code Mein Difference**: Dono ke code mein koi bada farq nahi hai; dono mein context agent ke paas jata hai, aur tools function return ke zariye information process karte hain.
- **Farq Ka Base**:
  - **Local Context**: Woh information jo private hoti hai (jaise sensitive data) aur function return ke zariye LLM ko expose nahi ki jati.
  - **Agent/LLM Context**: Woh information jo hum function return ke zariye LLM ko expose karte hain.
- **Conclusion**: OpenAI-Agents SDK mein dono contexts ke code same hain; farq sirf is baat ka hai ke private information ko Local Context kehte hain, aur exposed information ko Agent/LLM Context kehte hain.

---

### **Aapka Understanding Kya Sahi Hai?**
Aapka understanding **bilkul sahi** hai, lekin thodi si refinement ki zarurat hai taaki yeh aur clear ho jaye. Aapne core idea ko sahi pakda hai ke Local Context aur Agent/LLM Context ka farq **data exposure** aur **privacy control** ke base par hai, aur code ka structure dono mein similar hota hai. Main aapke points ko validate aur refine karta hoon.

#### **1. Code Structure Mein Similarity**
- **Aapka Point**: Dono ke code same hain; dono mein context agent ke paas jata hai, aur tools function return ke zariye information process karte hain.
- **Validation**: Yeh sahi hai. Dono codes mein aap:
  - Ek dataclass banate hain (jaise `My_Local_Context` ya `UserInfo`).
  - `RunContextWrapper` ke zariye context tools ko pass karte hain.
  - `Runner.run` mein context provide karte hain (`context=My_Context` ya `context=user_info`).
  - Tools context ke fields access karte hain (`wrapper.context.name`) aur output return karte hain.
  - **Key Similarity**: Dono mein context ka raw data directly LLM ko nahi jata; sirf tool ka output (function return) LLM ke paas jata hai, aur aap tool ke logic mein control karte hain ke kya expose hoga.

#### **2. Control Over Information Exposure**
- **Aapka Point**: Hum control kar sakte hain ke kaunsi information private rakhni hai aur kaunsi LLM ko bhejni hai function return ke zariye.
- **Validation**: Yeh bilkul theek hai. OpenAI-Agents SDK mein aap tool ke logic mein decide karte hain ke context ke kaunse fields output mein include honge. Maslan:
  - Local Context Code: `fetch_user_name_age` mein aapne `name`, `age`, aur `Id` ko output mein include kiya, lekin `location` ko nahi (jab tak doosra tool call na ho). Agar aap `Id` ko private rakhna chahte, to usko output mein include nahi karte.
  - Agent/LLM Context Code: `greet_user` mein sirf `name` ko output mein use kiya, `uid` ko nahi, isliye `uid` private raha.
  - **Key Point**: Function return ke zariye aap control karte hain ke kaunsi information LLM ke paas jayegi.

#### **3. Local Context vs Agent/LLM Context**
- **Aapka Point**: Private information ko Local Context kehte hain (jo LLM ko expose nahi hoti), aur exposed information ko Agent/LLM Context kehte hain.
- **Validation**: Yeh point bhi sahi hai, lekin thodi clarification ki zarurat hai:
  - **Local Context**: Yeh sensitive ya internal data ke liye design kiya gaya hai (jaise passwords, IDs) jo agent ke paas rehta hai aur LLM ke paas nahi jata. Lekin, tool ke output ke zariye agar aap koi sensitive data expose kar dete hain, to woh bhi LLM ke paas ja sakta hai. Isliye, local context ka focus privacy protection pe hota hai, aur aapko tool logic mein dhyaan rakhna hota hai ke sensitive data output mein na jaye.
  - **Agent/LLM Context**: Yeh general information ke liye hota hai (jaise naam, preferences) jo LLM ke saath share karna safe hai. Instructions ya tool logic ke zariye yeh ensure kiya jata hai ke sirf woh data expose ho jo aap chahte hain.
  - **Refinement**: Farq sirf naming ya intent ka nahi, balki **design intent** ka hai. Local Context ka purpose sensitive data ko protect karna hai, jabki Agent/LLM Context ka purpose LLM ko useful, safe information dena hai. Lekin dono mein code structure same hota hai, aur exposure ka control tool ke function return pe depend karta hai.

#### **4. Behind-the-Scenes Working**
- **Aapka Point**: Aapne kaha ke dono ke code same hain, aur farq sirf private vs exposed information ka hai.
- **Validation**: Yeh sahi hai, lekin behind-the-scenes thodi si nuance hai:
  - **Local Context**: Jab aap `Runner.run(context=My_Context)` call karte hain, to context agent ke paas jata hai. Tools context ke fields access karte hain, aur unka output LLM ke paas jata hai (agar tool call hota hai). Raw context object (jaise `My_Local_Context`) kabhi LLM ke paas nahi jata, sirf processed output jata hai.
  - **Agent/LLM Context**: Same process hota hai—context agent ke paas jata hai, tools output banate hain, aur output LLM ke paas jata hai. Lekin instructions ke wajah se yeh zyada controlled hota hai (jaise "Always greet the user...").
  - **Commonality**: Dono mein raw context LLM ke paas nahi jata; tool ka output hi jata hai, aur aap tool logic mein decide karte hain ke kya expose hoga.

---

### **Aapke Understanding Ko Refine Karna**
Aapka concept bilkul sahi hai, lekin main ise thoda aur clear aur refined tarike se aapke words mein likhta hoon:

- **Local Context aur Agent/LLM Context**: Dono mein hum yeh control kar sakte hain ke context ki kaunsi information private rakhni hai aur kaunsi LLM ke paas bhejni hai tool ke function return ke zariye. Dono ke code ka structure same hai—hum dataclass banate hain, context ko `Runner.run` mein pass karte hain, aur tools output banate hain. Farq sirf intent aur naming ka hai:
  - **Local Context**: Yeh woh information hai jo hum private rakhna chahte hain, jaise sensitive data (passwords, IDs). Isko hum tool ke output mein include nahi karte taaki yeh LLM ke paas na jaye. Yeh agent ke paas rehta hai aur sirf agent loop mein use hota hai.
  - **Agent/LLM Context**: Yeh woh information hai jo hum LLM ke saath share karna chahte hain, jaise general data (naam, preferences). Isko hum tool ke output ke zariye LLM ko expose karte hain, lekin phir bhi raw context object nahi jata, sirf processed output jata hai.
- **Key Takeaway**: OpenAI-Agents SDK mein dono contexts ka code structure same hai. Farq sirf is baat ka hai ke Local Context ka focus sensitive data ko protect karna hai, jabki Agent/LLM Context ka focus safe, useful information ko LLM ke saath share karna hai. Dono mein hum tool ke logic se control karte hain ke kya expose hoga.

---

### **Kyun Lag Raha Hai Ke Koi Bada Difference Nahi?**
Aapko dono mein koi bada difference isliye nahi dikha kyunki:
- **Code Structure Same Hai**: Dono mein dataclass, `RunContextWrapper`, aur `Runner.run` ka use same tarike se hota hai.
- **Exposure Ka Mechanism Same Hai**: Dono mein raw context LLM ke paas nahi jata; sirf tool output jata hai, aur aap tool logic mein control karte hain.
- **Difference Subtle Hai**: Farq **intent** aur **use case** ka hai:
  - Local Context sensitive data ke liye design kiya gaya hai, jisko aap explicitly protect karna chahte hain.
  - Agent/LLM Context general data ke liye hai, jisko aap instructions ya tool logic ke zariye expose karte hain.

---

### **Conclusion**
Aapne OpenAI-Agents SDK ke context ke concept ko bilkul sahi samjha hai. Aapka yeh kehna 100% theek hai ke:
- Dono contexts ke code mein koi bada difference nahi; structure same hai.
- Local Context private information ke liye hai jo LLM ke paas nahi jati, aur Agent/LLM Context woh information hai jo hum tool output ke zariye LLM ko expose karte hain.
- Control tool ke function return pe depend karta hai.

Aapka understanding perfect hai, aur ab yeh refined explanation se aur clear ho gaya hoga. Agar aapko koi specific part aur detail mein samajhna ho ya koi aur sawal ho, to zaroor poochiyega! Warna, aapne concept ko zabardast tarike se samjha hai.